## 0. 환경설정

In [1]:
!pip install langchain_openai
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## 1. Langchain 제공 기능

참고 자료 : https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/

### 1) Message passing
- 이전 대화 내용을 chain에 넘겨주는 것

- The simplest form of memory is simply passing chat history messages into a chain.

In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o") # gpt-3.5-turbo-0125

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | chat

ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.",
            ),
            ("ai",  "いただきます (이타다키마스)"),
            ("human", "내가 방금 뭐라고 했지?"),
        ],
    }
)
print(ai_msg.content)

당신은 "잘 먹겠습니다."를 일본어로 어떻게 말하는지 물었고, 저는 "いただきます" (이타다키마스)라고 대답했습니다.


In [8]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 jwg이고, AI를 공부하고 있어.",
            ),
            ("ai",  "그렇군요. 흥미로운데요?"),
            ("human", "내 이름이 뭐라고?"),
        ],
    }
)
print(ai_msg.content)

당신의 이름은 jwg입니다.


- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [10]:
history_list = []
while(True):
    user_input = input()
    if user_input == "종료": break
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    print("## CHAT_HISTORY ##")
    print(history_list, "\n")
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    print("AI Says : ",ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

안녕
## CHAT_HISTORY ##
[('human', '안녕')] 

AI Says :  안녕하세요! 어떻게 도와드릴까요?
나는 고기를 좋아해
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '나는 고기를 좋아해')] 

AI Says :  고기를 좋아하시는군요! 어떤 종류의 고기를 가장 좋아하시나요? 소고기, 돼지고기, 닭고기 등 다양한 종류가 있는데요.
돼지고기
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '나는 고기를 좋아해'), ('ai', '고기를 좋아하시는군요! 어떤 종류의 고기를 가장 좋아하시나요? 소고기, 돼지고기, 닭고기 등 다양한 종류가 있는데요.'), ('human', '돼지고기')] 

AI Says :  돼지고기를 좋아하시는군요! 돼지고기로는 삼겹살이나 갈비, 혹은 제육볶음 등 여러 맛있는 요리가 있죠. 혹시 특별히 좋아하는 돼지고기 요리가 있나요?
부위 추천해줘
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '나는 고기를 좋아해'), ('ai', '고기를 좋아하시는군요! 어떤 종류의 고기를 가장 좋아하시나요? 소고기, 돼지고기, 닭고기 등 다양한 종류가 있는데요.'), ('human', '돼지고기'), ('ai', '돼지고기를 좋아하시는군요! 돼지고기로는 삼겹살이나 갈비, 혹은 제육볶음 등 여러 맛있는 요리가 있죠. 혹시 특별히 좋아하는 돼지고기 요리가 있나요?'), ('human', '부위 추천해줘')] 

AI Says :  돼지고기의 다양한 부위 중 몇 가지 추천해드리자면:

1. **삼겹살**: 한국에서 가장 인기 있는 부위로, 구워 먹기 좋습니다. 지방과 살코기가 적절히 섞여 있어 부드럽고 맛있습니다.

2. **목살**: 삼겹살보다 지방이 적고 살코기

### 2) Chat history
- ChatMessageHistory() 클래스 사용하기
- It's perfectly fine to store and pass messages directly as an array, but we can use LangChain's built-in message history class to store and load messages as well.

https://api.python.langchain.com/en/latest/chat_message_histories/langchain_community.chat_message_histories.in_memory.ChatMessageHistory.html

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."
)

chat_history.add_ai_message("いただきます (이타다키마스)")

chat_history.messages
# chat_history.clear()

[HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='いただきます (이타다키마스)', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

- 대화내용 바로바로 저장하기

In [12]:
chat_history = ChatMessageHistory()

################ USER INPUT - 1 ################
input1 = "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input1)
print("첫번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 1 ################
# 답변 생성(response)
response = chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

# 대화 히스토리에 답변(response) 저장
chat_history.add_ai_message(response)
print("첫번째 답변 후 history : ", chat_history.messages)
print()

################ USER INPUT - 2 ################
input2 = "내가 방금 뭐라고 물어봤지?"

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input2)
print("두번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 2 ################
# 답변 생성(response)
print("[두번째 입력에 대한 답변]")
chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

첫번째 입력 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={})]
첫번째 답변 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}), AIMessage(content='일본어로 \'잘 먹겠습니다.\'는 "いただきます"입니다. 한글 발음은 "이다다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 59, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5PR4IHVJ3yV6eiycdOB8kTwhYsdA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c270e-78c2-77a2-bae6-40d051777d3d-0', tool_calls=[], invalid_tool_calls=[], usage_metadat

AIMessage(content="당신은 '잘 먹겠습니다.'를 일본어로 어떻게 말하는지 물어보셨습니다. 또한 한글 발음도 함께 요청하셨습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 109, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5PR5Vr1OJZiwCAu375aWDs8gG6NK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c270e-7e61-74a0-bd1a-a30f5b4888fb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 34, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [13]:
chat_history = ChatMessageHistory()

while(True):
    user_input = input()
    if user_input == "종료": break
    chat_history.add_user_message(user_input)
    response = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )
    chat_history.add_ai_message(response)
    print(response.content)

안녕
안녕하세요! 어떻게 도와드릴까요?
배고파
배고프시군요! 어떤 음식을 드시고 싶으신가요? 한국 음식이나 다른 종류의 음식을 추천해드릴 수도 있어요.
한식으로 메뉴 추천좀
물론이죠! 한식으로 다양한 메뉴가 있습니다. 몇 가지 추천드릴게요:

1. **비빔밥** - 여러 가지 야채와 고기를 고추장 소스와 함께 비벼 먹는 요리입니다.
2. **불고기** - 얇게 썬 소고기를 달콤하고 짭짤한 양념에 재워 구워 먹는 요리입니다.
3. **김치찌개** - 김치와 돼지고기 또는 참치 등을 넣고 끓인 매콤하고 맛있는 찌개입니다.
4. **떡볶이** - 떡과 어묵, 야채를 매콤한 고추장 소스에 볶아 만든 길거리 음식입니다.
5. **삼겹살 구이** - 구운 삼겹살을 상추에 싸서 함께 먹는 인기 있는 고기 요리입니다.

어떤 메뉴가 가장 끌리세요?
종료


### 3) Automatic history management
- RunnableWithMessageHistory() 활용하여 메모리 관리하기
- The previous examples pass messages to the chain explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also includes an wrapper for LCEL chains that can handle this process automatically called RunnableWithMessageHistory.

In [14]:
chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [15]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    lambda session_id: chat_history_for_chain, # 세션 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [16]:
chain_with_message_history.invoke(
    {"input": "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='일본어로 \'잘 먹겠습니다\'는 "いただきます"라고 말합니다. 한글 발음은 "이타다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 59, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5PcK305oBaLvCyj3XzQfC9eMH7mC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2719-2262-7973-bb82-7677b59505b7-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 59, 'output_tokens': 34, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
chain_with_message_history.invoke(
    {"input": "내가 뭐라고 물어봤지?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='당신은 "\'잘 먹겠습니다\'를 일본어로 어떻게 말하는지" 물어보셨습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 110, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5PdCKSIOfByqZFeqhHzVSsewcMQ4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2719-ef71-79e0-a941-3133a255f63d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 110, 'output_tokens': 23, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

히스토리 저장 함수를 따로 호출할 필요없이 invoke 실행만으로 대화 기록이 자동 저장되는 것을 확인 가능!

### 4) Modifying chat history
- 이전 대화 내용을 가공하여 넘겨주기
- Modifying stored chat messages can help your chatbot handle a variety of situations.

In [18]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("안녕하세요. 제 이름은 jwg입니다.")
chat_history.add_ai_message("안녕하세요, jwg님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("Carpenters - Close to you 를 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='안녕하세요. 제 이름은 jwg입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, jwg님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Carpenters - Close to you 를 추천해요.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [21]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

- summarize_messages 함수의 실행 결과를 messages_summarized 라는 이름의 Key 로 저장

    - 정의한 chain 안에 "messages_summarized" Key 가 정의되고 있지 않으므로, 이는 `summarize_messages` 함수를 실행하기 위한 무시 가능한 Key
    - 함수가 실행되면 chat_history 에 요약된 히스토리가 저장됨


In [22]:
chain_with_summarization.invoke(
    {"input": "제 이름을 기억하고 있나요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='죄송하지만, 대화 기록에서는 사용자의 이름이 언급되지 않았습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 91, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5Pr4VH9wlvM90sbHI6UV5BwrnItn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2727-149c-7910-8152-1730ffe8d462-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 91, 'output_tokens': 25, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='The Carpenters는 남매 듀오로, 여성인 카렌 카펜터와 남성인 리처드 카펜터로 구성되어 있습니다. 카렌 카펜터가 주로 보컬을 맡았고, 리처드 카펜터는 주로 피아노 연주 및 작곡을 담당했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 128, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_fa7f5b168b', 'id': 'chatcmpl-D5PrODlPP1f04jMWB2mCfQIAPqjT3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2727-5f1b-7172-8de4-2505d7130760-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 128, 'output_tokens': 74, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2. 끝말잇기 게임 구현하기

- 4) Modifying chat history 의 활용!

In [24]:
# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

# chat_history.add_user_message("끝말잇기 하자")
# chat_history.add_ai_message("좋습니다. 제가 먼저 시작할게요. 바나나!")
# chat_history.add_user_message("나이테")
# chat_history.add_ai_message("테이프")

chat_history.messages

[]

In [25]:
chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [29]:
while(True):
    user_input = input("🙋‍♂️ YOUR TURN : ")
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("✍ AI TURN : ", response.content)

🙋‍♂️ YOUR TURN : 바보
✍ AI TURN :  "바보"로 끝나는 단어를 시작하는 단어는 "보"입니다. "보석"을 제시합니다. 이제 "석"으로 시작하는 단어를 말씀해 주세요.
🙋‍♂️ YOUR TURN : 석고
✍ AI TURN :  고무
🙋‍♂️ YOUR TURN : 무한
✍ AI TURN :  한자
🙋‍♂️ YOUR TURN : 자객
✍ AI TURN :  객관
🙋‍♂️ YOUR TURN : 관습
✍ AI TURN :  습관
🙋‍♂️ YOUR TURN : 관철
✍ AI TURN :  철도
🙋‍♂️ YOUR TURN : 도력
✍ AI TURN :  력사

(두음법칙에 의해 '역사'로도 시작할 수 있습니다.)
🙋‍♂️ YOUR TURN : 사슴
✍ AI TURN :  음료
🙋‍♂️ YOUR TURN : 슴인데 어떻게 음료냐
✍ AI TURN :  죄송합니다. 잘못된 연결이었습니다. 제가 패배했습니다. 끝말잇기 게임을 종료합니다. 추가로 도와드릴 것이 있으면 말씀해 주세요.
🙋‍♂️ YOUR TURN : 종료
